**Scraping Laureados (selenium)**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://es.wikipedia.org/wiki/Anexo:Laureados_de_los_Premios_Nobel_por_pa%C3%ADs") 

paises = driver.find_elements(By.CLASS_NAME, 'mw-heading.mw-heading3')
with open("laureados.csv", "w", encoding="utf-8", newline='') as f:
    f.write('pais;nombre;categoria;anyo'+'\n')
    for pais in paises:
        pais_imp = (pais.text[:-8])
        div_pais = pais.find_element(By.XPATH, "./following-sibling::ol[1]") # following-sibling::ol coge todos los ol que siguen al nombre del país
                                                                                # así que añadimos[1] para que coja solo el primero de ellos
        filas = div_pais.find_elements(By.TAG_NAME, "li") 
        for i in filas:
            i = i.text.replace('-',',')
            i = i.split(',')
            for j in i[1:]:
                j.replace(' ', '')
            if len(i) > 3:
                i = [i[0], i[-2], i[-1]]
            if 'Paz' not in i[1]:
                texto = (pais_imp + ';' + ';'.join(i))
                texto = texto.replace('*','')
                f.write(texto + '\n')
        
    

**Scraping de gasto en educación (selenium y filtrado con re)**

In [2]:

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

driver = webdriver.Chrome()
driver.get("https://datosmacro.expansion.com/estado/gasto/educacion") 
driver.find_element(By.ID, 'ue-disagree-notice-button').click()

with open("gasto_educacion2.csv", "w", encoding="utf-8", newline='') as f:
    f.write('pais;anyo;gasto_pib'+'\n')
    while True:
        try: # este try es para salir del bucle cuando ya no queden anyos que revisar
            tabla = driver.find_element(By.ID, 'tb1')
            texto = tabla.find_elements(By.TAG_NAME, 'tr')
            titulos = texto[0]
            cuerpo = texto[1:]
            for linea in cuerpo:
                try: # este try es para aquellos datos que no aparezcan (cuando no esté la variable anyo)
                    nombre = re.findall(r'[A-ZÁÉÍÓÚ].+[a-záéíúó]', linea.text)
                    anyo = linea.find_element(By.CLASS_NAME, 'fecha')
                    numeros = linea.find_elements(By.CLASS_NAME, 'numero')
                    gasto_PIB = numeros[-4]
                    f.write(f'{nombre[0]};{anyo.text};{gasto_PIB.text}\n')
                except:
                    pass
            driver.find_element(By.CLASS_NAME, 'float-start.padleft').click()
        except:
            break
        

**Scraping de gasto en investigación (GET HTTP y normalización de nombres)**

In [ ]:
import requests

paises = {"Germany": "Alemania", 
          "Argentina": "Argentina",
          "Australia": "Australia",
          "Austria": "Austria",
          "Azerbaijan":"Azerbaiyán",
          "Belgium": "Bélgica",
          "Belarus": "Bielorrusia",
          "Bosnia and Herzegovina": "Bosnia y Herzegovina",
          "Bulgaria": "Bulgaria",
          "Canada": "Canadá",
          "Czechia":"República Checa",
          "Chile":"Chile",
          "China":"República Popular China",
          "Cyprus":"Chipre",
          "Colombia":"Colombia",
          "South Korea":"Corea del Sur", 
          "Croatia":"Croacia",
          "Denmark":"Dinamarca", 
          "Egypt":"Egipto",
          "Slovenia":"Eslovenia",
          "Spain":"España",
          "United States": "Estados Unidos", 
          "Faroe Islands":"Islas Feroe",
          "Finland":"Finlandia",
          "France":"Francia", 
          "Greece":"Grecia",
          "Guatemala":"Guatemala", 
          "Hong Kong":"Hong Kong",
          "Hungary":"Hungría",
          "India":"India",
          "Ireland":"Irlanda",
          "Israel":"Israel",
          "Italy":"Italia",
          "Iceland":"Islandia",
          "Japan":"Japón",
          "Lithuania":"Lituania",
          "Luxembourg":"Luxemburgo",
          "Mexico": "México",
          "Nigeria":"Nigeria",
          "Norway":"Noruega", 
          "New Zealand":"Nueva Zelanda",
          "Netherlands":"Países Bajos",
          "Pakistan":"Pakistán",
          "Peru":"Perú",
          "Poland":"Polonia",
          "Portugal":"Portugal",
          "United Kingdom":"Reino Unido",
          "Romania":"Rumania",
          "Russia":"Rusia y Unión Soviética",
          "Saint Lucia": "Santa Lucía",
          "South Africa":"Sudáfrica",
          "Sweden":"Suecia",
          "Switzerland":"Suiza",
          "Trinidad and Tobago": "Trinidad y Tobago",
          "Turkey":"Turquía",
          "Ukraine":"Ucrania",
          "Venezuela":"Venezuela"}

url = "https://ourworldindata.org/grapher/research-spending-gdp.csv"
texto = requests.get(url).text
l = texto.split("\n") 

# filtramos solo los paises que ya aparecen en laureados
# traduciéndolos y guardándonos los datos en un nuevo csv

with open("research_spending.csv", "w", encoding="utf-8") as f:
    f.write("pais;codigo;anyo;investigacion_pib\n")
    print("pais;codigo;anyo;investigacion_pib")
    for linea in l:
        for en, es in paises.items():
            if linea.startswith(en + ","):
                linea = linea.replace(en, es, 1)
                linea = linea.replace(",", ";") 
                print(linea)
                f.write(linea + "\n")


pais;codigo;anyo;investigacion_pib
Argentina;ARG;1996;0.41749
Argentina;ARG;1997;0.41959
Argentina;ARG;1998;0.41131
Argentina;ARG;1999;0.45337
Argentina;ARG;2000;0.43884
Argentina;ARG;2001;0.42461
Argentina;ARG;2002;0.38886
Argentina;ARG;2003;0.41013
Argentina;ARG;2004;0.40376
Argentina;ARG;2005;0.42074
Argentina;ARG;2006;0.45216
Argentina;ARG;2007;0.46007
Argentina;ARG;2008;0.47055
Argentina;ARG;2009;0.58398
Argentina;ARG;2010;0.56104
Argentina;ARG;2011;0.56597
Argentina;ARG;2012;0.63491
Argentina;ARG;2013;0.61849
Argentina;ARG;2014;0.59396
Argentina;ARG;2015;0.62262
Argentina;ARG;2016;0.55815
Argentina;ARG;2017;0.55631
Argentina;ARG;2018;0.4883
Argentina;ARG;2019;0.47813
Argentina;ARG;2020;0.54126
Argentina;ARG;2021;0.52216
Argentina;ARG;2022;0.54947
Australia;AUS;1996;1.66218
Australia;AUS;1998;1.51351
Australia;AUS;2000;1.57198
Australia;AUS;2002;1.74654
Australia;AUS;2004;1.84802
Australia;AUS;2006;2.17862
Australia;AUS;2008;2.39898
Australia;AUS;2010;2.37009
Australia;AUS;2011;2.